In [45]:
DATASET_DIR = '/home/crueang/Chaks/AIOT_project/data/Office-31'
HOME = '/home/crueang/Chaks/AIOT_project'
CP = '/home/crueang/Chaks/AIOT_project/democp'

NUM_CLASSES = 31
BATCH_SIZE = 16
IMAGE_SIZE = 224

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision.models import mobilenet_v3_large, mobilenet_v3_small
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchsummary import summary
import torchvision
from torch.utils.data import Dataset
device = "cuda" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose([
    torchvision.models.MobileNet_V3_Small_Weights.IMAGENET1K_V1.transforms(),
])

amazon_dataset = ImageFolder(root=f'{DATASET_DIR}/amazon', transform=transform)
dslr_dataset = ImageFolder(root=f'{DATASET_DIR}/dslr', transform=transform)
webcam_dataset = ImageFolder(root=f'{DATASET_DIR}/webcam', transform=transform)

classes = amazon_dataset.classes

print(classes)

print(len(amazon_dataset), len(dslr_dataset), len(webcam_dataset))


['back_pack', 'bike', 'bike_helmet', 'bookcase', 'bottle', 'calculator', 'desk_chair', 'desk_lamp', 'desktop_computer', 'file_cabinet', 'headphones', 'keyboard', 'laptop_computer', 'letter_tray', 'mobile_phone', 'monitor', 'mouse', 'mug', 'paper_notebook', 'pen', 'phone', 'printer', 'projector', 'punchers', 'ring_binder', 'ruler', 'scissors', 'speaker', 'stapler', 'tape_dispenser', 'trash_can']
2817 498 795


In [47]:
# kd
temperature, alpha, l2_weight = 3, 0.9, 0

# mmd
mmd_alpha = 1

# coral
lambda_coral = 1

# pixel mix
use_pixel_mix = False

# src
src_weight = 0.8

In [48]:
# @title FLOPS computation
# Code from https://github.com/Eric-mingjie/rethinking-network-pruning/blob/master/imagenet/l1-norm-pruning/compute_flops.py
import numpy as np
import os

import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable


def print_model_param_nums(model=None):
    if model == None:
        model = torchvision.models.alexnet()
    total = sum([param.nelement() if param.requires_grad else 0 for param in model.parameters()])
    print('  + Number of params: %.4fM' % (total / 1e6))

def count_model_param_flops(model=None, input_res=224, multiply_adds=True, device='cpu'):

    prods = {}
    def save_hook(name):
        def hook_per(self, input, output):
            prods[name] = np.prod(input[0].shape)
        return hook_per

    list_1=[]
    def simple_hook(self, input, output):
        list_1.append(np.prod(input[0].shape))
    list_2={}
    def simple_hook2(self, input, output):
        list_2['names'] = np.prod(input[0].shape)


    list_conv=[]
    def conv_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size[0] * self.kernel_size[1] * (self.in_channels / self.groups)
        bias_ops = 1 if self.bias is not None else 0

        params = output_channels * (kernel_ops + bias_ops)
        # flops = (kernel_ops * (2 if multiply_adds else 1) + bias_ops) * output_channels * output_height * output_width * batch_size

        num_weight_params = (self.weight.data != 0).float().sum()
        flops = (num_weight_params * (2 if multiply_adds else 1) + bias_ops * output_channels) * output_height * output_width * batch_size

        list_conv.append(flops)

    list_linear=[]
    def linear_hook(self, input, output):
        batch_size = input[0].size(0) if input[0].dim() == 2 else 1

        weight_ops = self.weight.nelement() * (2 if multiply_adds else 1)
        bias_ops = self.bias.nelement()

        flops = batch_size * (weight_ops + bias_ops)
        list_linear.append(flops)

    list_bn=[]
    def bn_hook(self, input, output):
        list_bn.append(input[0].nelement() * 2)

    list_relu=[]
    def relu_hook(self, input, output):
        list_relu.append(input[0].nelement())

    list_pooling=[]
    def pooling_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        kernel_ops = self.kernel_size * self.kernel_size
        bias_ops = 0
        params = 0
        flops = (kernel_ops + bias_ops) * output_channels * output_height * output_width * batch_size

        list_pooling.append(flops)

    list_upsample=[]

    # For bilinear upsample
    def upsample_hook(self, input, output):
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()

        flops = output_height * output_width * output_channels * batch_size * 12
        list_upsample.append(flops)

    def foo(net):
        childrens = list(net.children())
        if not childrens:
            if isinstance(net, torch.nn.Conv2d):
                net.register_forward_hook(conv_hook)
            if isinstance(net, torch.nn.Linear):
                net.register_forward_hook(linear_hook)
            if isinstance(net, torch.nn.BatchNorm2d):
                net.register_forward_hook(bn_hook)
            if isinstance(net, torch.nn.ReLU):
                net.register_forward_hook(relu_hook)
            if isinstance(net, torch.nn.MaxPool2d) or isinstance(net, torch.nn.AvgPool2d):
                net.register_forward_hook(pooling_hook)
            if isinstance(net, torch.nn.Upsample):
                net.register_forward_hook(upsample_hook)
            return
        for c in childrens:
            foo(c)

    if model == None:
        model = torchvision.models.alexnet()
    foo(model)
    input = Variable(torch.rand(3,input_res,input_res).unsqueeze(0), requires_grad = True).to(device)
    out = model(input)


    total_flops = (sum(list_conv) + sum(list_linear) + sum(list_bn) + sum(list_relu) + sum(list_pooling) + sum(list_upsample))

    print('Number of FLOPs: %.6f GFLOPs (%.2f MFLOPs)' % (total_flops / 1e9, total_flops / 1e6))

    return total_flops

In [49]:
resume_path = f'{CP}/pretrained_mmd25/best_model.pth'

In [50]:
teacher_model = torch.load('/home/crueang/Chaks/AIOT_project/best_resnet_amazon.pth', weights_only=False, map_location=device)
teacher_model.eval()
model = torchvision.models.mobilenet_v3_small(weights=None, width_mult=0.25, num_classes=31).to(device)
if resume_path:
    model.load_state_dict(torch.load(resume_path, weights_only=True, map_location=device))
summary(model, (3, 224, 224))
count_model_param_flops(model=model.cpu().eval(), input_res=224, multiply_adds=True)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 112, 112]             216
       BatchNorm2d-2          [-1, 8, 112, 112]              16
         Hardswish-3          [-1, 8, 112, 112]               0
            Conv2d-4            [-1, 8, 56, 56]              72
       BatchNorm2d-5            [-1, 8, 56, 56]              16
              ReLU-6            [-1, 8, 56, 56]               0
 AdaptiveAvgPool2d-7              [-1, 8, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]              72
              ReLU-9              [-1, 8, 1, 1]               0
           Conv2d-10              [-1, 8, 1, 1]              72
      Hardsigmoid-11              [-1, 8, 1, 1]               0
SqueezeExcitation-12            [-1, 8, 56, 56]               0
           Conv2d-13            [-1, 8, 56, 56]              64
      BatchNorm2d-14            [-1, 8,

tensor(18659808.)

In [51]:
def freeze_before(model, n):
    for name, param in model.named_parameters():
        if 'features' in name and int(name.split('.')[1]) < n:
            param.requires_grad = False
        else:
            param.requires_grad = True
        print(f'Layer: {name}, requires_grad: {param.requires_grad}')

In [52]:
class DistillationLoss(nn.Module):
    def __init__(self, temperature=1.0, alpha=0.9, l2_weight=0.01):
        super(DistillationLoss, self).__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.l2_weight = l2_weight
        self.criterion_soft = nn.KLDivLoss(reduction='batchmean')  # Soft targets
        self.criterion_hard = nn.CrossEntropyLoss()  # Hard targets

    def forward(self, student_logits, teacher_logits, labels, student_params, teacher_params):
        # Soft loss (knowledge distillation)
        soft_student_probs = torch.log_softmax(student_logits / self.temperature, dim=1)
        soft_teacher_probs = torch.softmax(teacher_logits / self.temperature, dim=1)
        soft_loss = self.criterion_soft(soft_student_probs, soft_teacher_probs)

        # Hard loss (ground truth labels)
        hard_loss = self.criterion_hard(student_logits, labels)

        # # L2 Loss to match student parameters to teacher's parameters
        l2_loss = 0.0
        for student_param, teacher_param in zip(student_params, teacher_params):
            try:
                l2_loss += torch.norm(student_param - teacher_param, p=2)
            except:
                pass

        # Total loss
        total_loss = (self.alpha * soft_loss + (1 - self.alpha) * hard_loss) + self.l2_weight * l2_loss
        return total_loss

In [53]:
class CorrelationAlignmentLoss(nn.Module):
    def __init__(self):
        super(CorrelationAlignmentLoss, self).__init__()

    def forward(self, f_s: torch.Tensor, f_t: torch.Tensor) -> torch.Tensor:
        mean_s = f_s.mean(0, keepdim=True)
        mean_t = f_t.mean(0, keepdim=True)
        cent_s = f_s - mean_s
        cent_t = f_t - mean_t
        cov_s = torch.mm(cent_s.t(), cent_s) / (len(f_s) - 1)
        cov_t = torch.mm(cent_t.t(), cent_t) / (len(f_t) - 1)

        mean_diff = (mean_s - mean_t).pow(2).mean()
        cov_diff = (cov_s - cov_t).pow(2).mean()

        return mean_diff + cov_diff

In [54]:
# mmd loss    
def guassian_kernel(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    n_samples = int(source.size()[0])+int(target.size()[0])
    total = torch.cat([source, target], dim=0)
    total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    L2_distance = ((total0-total1)**2).sum(2)
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]
    return sum(kernel_val)#/len(kernel_val)

def mmdloss(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    batch_size = int(source.size()[0])
    kernels = guassian_kernel(source, target,
                              kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)
    XX = kernels[:batch_size, :batch_size]
    YY = kernels[batch_size:, batch_size:]
    XY = kernels[:batch_size, batch_size:]
    YX = kernels[batch_size:, :batch_size]
    loss = torch.mean(XX + YY - XY -YX)
    return loss

In [55]:
# pixel mix
import random
def pixel_mix(source, target, alpha=0.9):
    # Get the dimensions of the images
    batch_size, _, height, width = target.size()

    # Create an empty tensor for mixed images
    mixed_images = torch.empty_like(target)

    for i in range(batch_size):
        for h in range(height):
            for w in range(width):
                # Randomly choose between source and target pixels
                if random.random() < alpha:
                    mixed_images[i, :, h, w] = source[i, :, h, w]
                else:
                    mixed_images[i, :, h, w] = target[i, :, h, w]

    return mixed_images

In [56]:
freeze_before(model, 0)

Layer: features.0.0.weight, requires_grad: True
Layer: features.0.1.weight, requires_grad: True
Layer: features.0.1.bias, requires_grad: True
Layer: features.1.block.0.0.weight, requires_grad: True
Layer: features.1.block.0.1.weight, requires_grad: True
Layer: features.1.block.0.1.bias, requires_grad: True
Layer: features.1.block.1.fc1.weight, requires_grad: True
Layer: features.1.block.1.fc1.bias, requires_grad: True
Layer: features.1.block.1.fc2.weight, requires_grad: True
Layer: features.1.block.1.fc2.bias, requires_grad: True
Layer: features.1.block.2.0.weight, requires_grad: True
Layer: features.1.block.2.1.weight, requires_grad: True
Layer: features.1.block.2.1.bias, requires_grad: True
Layer: features.2.block.0.0.weight, requires_grad: True
Layer: features.2.block.0.1.weight, requires_grad: True
Layer: features.2.block.0.1.bias, requires_grad: True
Layer: features.2.block.1.0.weight, requires_grad: True
Layer: features.2.block.1.1.weight, requires_grad: True
Layer: features.2.bl

In [57]:
train_loader = DataLoader(amazon_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dslr_dataset, batch_size=16, shuffle=False)

criterion = nn.CrossEntropyLoss()
distillation_loss = DistillationLoss(temperature, alpha, l2_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

checkpoint_path = f'{CP}/pretrained_mmd26/'

In [61]:
# train_student_2
from tqdm import tqdm
def train_da(teacher_model, student_model, opt, loss_fn, distillation_loss, Dl_source,Dl_target ,test_loader, epochs=10, checkpoint_path=None, device='cpu'):
    training_logs = {"train_loss": [], "validate_loss": [], "train_acc": [], "validate_acc": []}
    epoch_number = 0
    best_test_loss = float('inf')
    student_model = student_model.to(device)
    # alpha_loss = 0.1
    if checkpoint_path:
      if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        
      if os.path.exists(checkpoint_path + 'model.pth'):
        student_model.load_state_dict(torch.load(checkpoint_path + 'model.pth', weights_only=True, map_location=device))

      if os.path.exists(checkpoint_path + 'opt.pth'):
        opt.load_state_dict(torch.load(checkpoint_path + 'opt.pth', weights_only=True, map_location=device))

      if os.path.exists(checkpoint_path + 'training_logs.pth'):
        training_logs = torch.load(checkpoint_path + 'training_logs.pth', weights_only=True)
        epoch_number = len(training_logs['train_loss'])
        best_test_loss = min(best_test_loss, min(training_logs['validate_loss']))
        print(max(training_logs['validate_acc']))

    for i in range(epoch_number):
        print(f"Epochs {i+1}".ljust(10), end='')
        for k, v in training_logs.items():
            print(f"{k}: {v[i]:.5f}", end=" ")
        print()

    print("🤖Training on", device)
    max_batch = min(len(Dl_source),len(Dl_target))
    for epoch in range(epoch_number, epochs):

        train_loss, train_correct, mmd = 0, 0, 0
        student_model.train()
        print(f'epochs {epoch+1:04d} / {epochs:04d}', end='\n============\n')
        Dl_source_iter = iter(Dl_source)
        Dl_target_iter = iter(Dl_target)
        p = (epoch + 1) / epochs
        
        for batch_idx in range(max_batch):
            
            opt.zero_grad()
            X_s, y_s = next(Dl_source_iter)

            X_t, y_t = next(Dl_target_iter)
            
            if use_pixel_mix:
              X_s = pixel_mix(X_s, X_t)

            if X_s.shape[0] != X_t.shape[0]:
              min_bs = min(X_s.shape[0], X_t.shape[0])
              X_s = X_s[:min_bs]
              y_s = y_s[:min_bs]
              X_t = X_t[:min_bs]
              y_t = y_t[:min_bs]
            
            X_s, y_s = X_s.to(device), y_s.to(device)
            X_t, y_t = X_t.to(device), y_t.to(device)

            class_pred_s = student_model(X_s)
            class_pred_t = student_model(X_t)

            loss_src = loss_fn(class_pred_s, y_s) 
            mmd_loss = mmdloss(class_pred_s, class_pred_t)*mmd_alpha
            
             # Knowledge distillation loss
            with torch.no_grad():
                teacher_output = teacher_model(X_s)
                
            student_params = list(model.parameters())
            teacher_params = list(teacher_model.parameters())
                
            loss_distillation = distillation_loss(class_pred_s, teacher_output, y_s, student_params, teacher_params)
            
            loss_coral = CorrelationAlignmentLoss()(class_pred_s, class_pred_t)*lambda_coral

            loss = loss_src*src_weight + mmd_loss + loss_distillation + loss_coral
            loss.backward()
            opt.step()

            train_loss += loss.item()
            mmd += mmd_loss.item()
            with torch.no_grad():
                class_prediction_S = student_model(X_s)
                train_correct_s = (class_prediction_S.argmax(1) == y_s).float().sum().item()
                class_prediction_T = student_model(X_t)
                train_correct_t = (class_prediction_T.argmax(1) == y_t).float().sum().item()
                
            train_correct += train_correct_s

        avg_train_loss = train_loss / (max_batch*BATCH_SIZE)
        avg_train_acc = train_correct / (max_batch*BATCH_SIZE)
        print(f'\n\tTrain loss: {avg_train_loss}')
        print(f'\tTrain acc: {avg_train_acc}')
        training_logs["train_loss"].append(avg_train_loss)
        training_logs["train_acc"].append(avg_train_acc)

        test_loss, test_correct = 0, 0
        student_model.eval()
        test_bar = tqdm(test_loader , desc='📄Testing' , unit='batch')
        with torch.no_grad():
          for images, label in test_bar:
            images = images.to(device)
            label = label.to(device)
            outputs = student_model(images)
            loss = loss_fn(outputs, label)

            test_loss += loss.item()
            test_correct += (outputs.argmax(1) == label).float().sum().item()

        avg_test_loss = test_loss / len(test_loader.dataset)
        avg_test_acc = test_correct / len(test_loader.dataset)
        print(f'\tTest loss: {avg_test_loss}')
        print(f'\tTest acc: {avg_test_acc}')
        training_logs["validate_loss"].append(avg_test_loss)
        training_logs["validate_acc"].append(avg_test_acc)

        if checkpoint_path:
            torch.save(student_model.state_dict(), checkpoint_path + "model.pth")
            torch.save(opt.state_dict(), checkpoint_path + "opt.pth")
            torch.save(training_logs, checkpoint_path + 'training_logs.pth')
            if best_test_loss > avg_test_loss:
               torch.save(student_model.state_dict(), checkpoint_path + "best_model.pth")
               best_test_loss = avg_test_loss


In [62]:
train_da(teacher_model, model, optimizer, criterion, distillation_loss, train_loader, test_loader, test_loader, 50, checkpoint_path=checkpoint_path, device=device)

0.9618473895582329
Epochs 1  train_loss: 1.55582 validate_loss: 0.09290 train_acc: 0.68164 validate_acc: 0.65663 
Epochs 2  train_loss: 1.47441 validate_loss: 0.08451 train_acc: 0.63672 validate_acc: 0.70080 
Epochs 3  train_loss: 1.39879 validate_loss: 0.07612 train_acc: 0.67188 validate_acc: 0.71084 
Epochs 4  train_loss: 1.32647 validate_loss: 0.07450 train_acc: 0.69531 validate_acc: 0.74498 
Epochs 5  train_loss: 1.25006 validate_loss: 0.06742 train_acc: 0.73633 validate_acc: 0.73896 
Epochs 6  train_loss: 1.20225 validate_loss: 0.07854 train_acc: 0.69531 validate_acc: 0.69679 
Epochs 7  train_loss: 1.16494 validate_loss: 0.06406 train_acc: 0.69531 validate_acc: 0.75703 
Epochs 8  train_loss: 1.10018 validate_loss: 0.05328 train_acc: 0.73242 validate_acc: 0.78112 
Epochs 9  train_loss: 1.05939 validate_loss: 0.04555 train_acc: 0.72070 validate_acc: 0.77711 
Epochs 10 train_loss: 1.02230 validate_loss: 0.04772 train_acc: 0.72852 validate_acc: 0.79317 
Epochs 11 train_loss: 0.97688 v